In [ ]:
import os
import geopandas as gpd
import rasterio
import rasterstats as rs
import pandas as pd
import numpy as np
from rasterstats import zonal_stats


Calculate Zonal Statistics

In [ ]:


# Define the input and output directories
input_folder = r".\CLM"
point_feature_class = r".\CLM_XYTableToPoint.shp"
output_folder = r".\CLM\Zonal_Stats"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Load the point feature class as a GeoDataFrame
gdf = gpd.read_file(point_feature_class)

# Ensure that the point dataset has a unique ID column
point_id_field = "pointid"  # Adjust if the field name differs

# List all TIFF files in the input folder
tiff_files = [f for f in os.listdir(input_folder) if f.endswith('.tif')]

# Loop through each TIFF file and calculate zonal statistics
for tiff_file in tiff_files:
    input_tiff = os.path.join(input_folder, tiff_file)
    
    # Compute zonal statistics
    zonal_stats = rs.zonal_stats(
        gdf, input_tiff, stats=["max"], geojson_out=True
    )

    # Convert results to DataFrame
    df = pd.DataFrame([
        {
            point_id_field: feature["properties"][point_id_field],
            "MaxValue": feature["properties"]["max"]
        }
        for feature in zonal_stats
    ])

    # Define the output file path (CSV instead of DBF)
    output_csv = os.path.join(output_folder, f"ZonalStats_{os.path.splitext(tiff_file)[0]}.csv")
    df.to_csv(output_csv, index=False)

    print(f"Zonal statistics calculated for {input_tiff} and saved to {output_csv}")

print("All zonal statistics have been calculated.")


In [ ]:
# Define the directory containing the CSV files
input_directory = r'.\CLM\Zonal_Stats'
output_directory = r'.\CLM\MSE_calculation'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Initialize an empty list to hold DataFrames
dataframes = []

# Loop through all files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith('.csv'):
        # Read the CSV file
        df = pd.read_csv(os.path.join(input_directory, filename))
        
        # Get the base name of the file (without extension)
        base_name = os.path.splitext(filename)[0]
        
        # Rename columns that contain 'MaxValue' to include the base name
        df.columns = df.columns.str.replace('MaxValue', f'{base_name}_Max', case=False)
        
        # Remove the specific substring from column names
        df.columns = df.columns.str.replace('ZonalStats_', '', case=False)
        
        # Drop columns that contain 'COUNT' or 'Area'
        df = df.loc[:, ~df.columns.str.contains('COUNT|AREA', case=False)]
        
        # Check for duplicate columns and keep the first occurrence
        df = df.loc[:, ~df.columns.duplicated(keep='first')]

        dataframes.append(df)

# Merge all DataFrames on 'pointid'
if dataframes:
    merged_df = dataframes[0]
    for df in dataframes[1:]:
        merged_df = pd.merge(merged_df, df, on='pointid', how='outer', suffixes=('', '_dup'))
        
        # Remove duplicate columns that may have been created during the merge
        merged_df = merged_df.loc[:, ~merged_df.columns.str.endswith('_dup')]

    # Save the merged DataFrame to a new Excel file in the output directory
    output_file_path = os.path.join(output_directory, 'merged_output.xlsx')
    merged_df.to_excel(output_file_path, index=False)

    print(f"Merging and cleaning completed. Output saved as '{output_file_path}'.")
else:
    print("No CSV files found to merge.")


Merging and cleaning completed. Output saved as 'C:\Users\jayja\Documents\ArcGIS\Projects\CLM_Testing\CLM\MSE calculation\merged_output.xlsx'.


Merge PFT to existing PFT list

In [ ]:
import pandas as pd
import os

# Define file paths
merged_output_path = r".\CLM\MSE calculation\merged_output.xlsx"
pfts_path = r".\CLM\MSE calculation\PFTs.xlsx"
output_csv_path = r".\CLM\MSE calculation\merged_result.csv"

try:
    # Load the data
    merged_output_df = pd.read_excel(merged_output_path)
    pfts_df = pd.read_excel(pfts_path)

    # Merge the data on 'pointid'
    merged_df = pd.merge(merged_output_df, pfts_df, on='pointid', how='outer')

    # Save the result to a new CSV file
    merged_df.to_csv(output_csv_path, index=False)
    print(f"Merged file successfully created at: {output_csv_path}")
except FileNotFoundError as fnf_error:
    print(f"File not found: {fnf_error}")
except Exception as e:
    print(f"An error occurred: {e}")


Merged file successfully created at: C:\Users\jayja\Documents\ArcGIS\Projects\CLM_Testing\CLM\MSE calculation\merged_result.csv


Tabulate the area

In [ ]:
# Define the file paths
workspace = r".\CLM_Testing\CLM_Testing.gdb"
input_raster = f"'{workspace}/CellSta_MSD_2008'"  # GDAL requires single quotes
vector_layer = os.path.join(workspace, "CLM_Reprojected_PointToRaster.shp")  # Assuming it's a shapefile
output_csv = os.path.join(workspace, "Tabulate_CLM_MSD_Classes.csv")

# Step 1: Get the cell size of the raster from GDB
try:
    with rasterio.open(f"gdal://{input_raster}") as raster:
        cell_size = raster.res[0]  # Assuming square pixels
        print(f"Processing cell size set to: {cell_size}")
except Exception as e:
    print(f"Error reading raster from GDB: {str(e)}")
    exit()

# Step 2: Tabulate Area (Zonal Statistics)
try:
    # Load the vector data (geospatial points or polygons)
    gdf = gpd.read_file(vector_layer)

    # Compute zonal statistics (area covered by each value in raster)
    stats = zonal_stats(gdf, f"gdal://{input_raster}", stats=["count"], categorical=True, geojson_out=True)

    # Convert results to a DataFrame
    tabulate_df = pd.DataFrame([
        {**{"ID": feature["properties"]["ID"]}, **feature["properties"]}
        for feature in stats
    ])

    # Save the output table
    tabulate_df.to_csv(output_csv, index=False)
    print("Tabulate Area completed successfully.")

except Exception as e:
    print(f"Error during zonal statistics calculation: {str(e)}")


Processing cell size set to: 30.0000016640515
Tabulate Area completed successfully.


Merge PFTs with Tabulated Area

In [ ]:
# File paths
output_folder = r".\CLM\MSE calculation"
exported_table = os.path.join(output_folder, "Tabulate_CLM_MSD_Classes.csv")
existing_table = os.path.join(output_folder, "merged_result.csv")
merged_output = os.path.join(output_folder, "final_merged_output.csv")

# Check if both tables exist
if os.path.exists(exported_table) and os.path.exists(existing_table):
    # Load the tables as pandas DataFrames
    df_exported = pd.read_csv(exported_table)
    df_existing = pd.read_csv(existing_table)

    # Merge the tables on 'pointid' (left) and 'Value' (right)
    df_merged = pd.merge(df_existing, df_exported, left_on='pointid', right_on='VALUE', how='outer')

    # Save the merged table to a new CSV file
    df_merged.to_csv(merged_output, index=False)
    print(f"Tables merged successfully into {merged_output}")
else:
    print(f"One or both tables not found. Ensure both '{exported_table}' and '{existing_table}' exist.")

Tables merged successfully into C:\Users\jayja\Documents\ArcGIS\Projects\CLM_Testing\CLM\MSE calculation\final_merged_output.csv


Rename Tabulated Area Values

In [ ]:
# File paths
output_folder = r".\CLM\MSE calculation"
merged_file = os.path.join(output_folder, "final_merged_output.csv")
classification_file = os.path.join(output_folder, "Raw Classification.csv")

# Check if the files exist
if os.path.exists(merged_file) and os.path.exists(classification_file):
    # Load the merged file and classification file into DataFrames
    df_merged = pd.read_csv(merged_file)
    df_classification = pd.read_csv(classification_file)
    
    # Extract the 'MSD Value' and 'Layer Name' columns from classification.csv
    # The 'MSD Value' will be used to map to 'VALUE_' columns in the merged file
    value_to_layer = {}
    
    # Iterate through rows in classification.csv to create the mapping from MSD Value to Layer Name
    for idx, row in df_classification.iterrows():
        msd_value = row['MSD Val']
        layer_name = row['Layer Name']
        value_to_layer[msd_value] = layer_name
    
    # Identify the 'VALUE_' columns in the merged DataFrame
    value_columns = [col for col in df_merged.columns if 'VALUE_' in col]
    
    # Create a dictionary to map the 'VALUE_' columns to the corresponding Layer Name
    value_column_mapping = {}
    
    for value_column in value_columns:
        # Extract the number from the 'VALUE_' column (e.g., 'VALUE_1' -> 1)
        value_number = int(value_column.split('_')[1])
        
        # Map the MSD Value to the corresponding Layer Name from classification.csv
        if value_number in value_to_layer:
            value_column_mapping[value_column] = value_to_layer[value_number]
    
    # Rename the columns in df_merged using the mapping
    df_merged.rename(columns=value_column_mapping, inplace=True)

    # Save the updated DataFrame back to CSV
    df_merged.to_csv(merged_file, index=False)
    print(f"Column names updated successfully in {merged_file}")
else:
    print(f"One or both files '{merged_file}' and '{classification_file}' do not exist.")

Column names updated successfully in C:\Users\jayja\Documents\ArcGIS\Projects\CLM_Testing\CLM\MSE calculation\final_merged_output.csv


Proportions

In [ ]:
# Define the file paths
input_file_path = r".\CLM\MSE calculation\final_merged_output.csv"
output_file_path = r".\CLM\MSE calculation\final_merge_output_MSD.csv"

# Load the input CSV file
data = pd.read_csv(input_file_path)

# Select the first column and all columns starting from "Irrigated Temperate Corn"
start_column = "Irrigated Temperate Corn"

# Get the list of all column names
columns = data.columns.tolist()

# Find the index of the starting column
start_index = columns.index(start_column)

# Slice the DataFrame to include the first column and columns from the starting column onward
filtered_data = pd.concat([data.iloc[:, 0], data.iloc[:, start_index:]], axis=1)

# Save the filtered data to a new CSV file
filtered_data.to_csv(output_file_path, index=False)

print(f"Filtered data saved to: {output_file_path}")

Filtered data saved to: C:\Users\jayja\Documents\ArcGIS\Projects\CLM_Testing\CLM\MSE calculation\final_merge_output_MSD.csv


In [ ]:
# Define the file path (update it if necessary)
file_path = r".\CLM\MSE calculation\final_merged_output_MSD.csv"

# Load the CSV file
data = pd.read_csv(file_path)

# Exclude the first column for calculations
first_column = data.iloc[:, 0]  # Save the first column
data_for_proportions = data.iloc[:, 1:]  # Select the rest of the columns

# Calculate proportions for each cell
proportions = data_for_proportions.div(data_for_proportions.sum(axis=1), axis=0)

# Reattach the first column
result = pd.concat([first_column, proportions], axis=1)

# Save the result to a new CSV file
output_path = r".\CLM\MSE calculation\final_merge_output_proportions.csv"
result.to_csv(output_path, index=False)

print(f"Proportions calculated and saved to: {output_path}")

Proportions calculated and saved to: C:\Users\jayja\Documents\ArcGIS\Projects\CLM_Testing\CLM\MSE calculation\final_merge_output_proportions.csv


Spatial Aggregations

In [ ]:
import pandas as pd

# File paths
proportions_file = r".\CLM\MSE calculation\final_merge_output_proportions.csv"
classification_file = r".CLM\MSE calculation\Raw Classification.csv"
output_file = r".\CLM\MSE calculation\aggregated_output.csv"

# Load the proportions data
proportions_df = pd.read_csv(proportions_file)

# Load the classification data from the Excel file
classification_df = pd.read_csv(classification_file)

# Ensure column names in classification data are stripped of extra spaces
classification_df['Layer Name'] = classification_df['Layer Name'].str.strip()

# Get the name of the first column in proportions_df (e.g., identifiers)
first_column_name = proportions_df.columns[0]

# Filter proportions columns to include only those matching Layer Name in the classification file
matching_columns = [first_column_name] + classification_df['Layer Name'].tolist()
filtered_proportions = proportions_df[proportions_df.columns.intersection(matching_columns)]

print("filtered proportions")
# Reshape the filtered proportions data for aggregation
melted_df = filtered_proportions.melt(id_vars=[first_column_name], var_name='Layer Name', value_name='Proportion')

# Merge melted proportions data with classification data
merged_df = pd.merge(melted_df, classification_df, on='Layer Name', how='inner')
print('finished merging')

# Aggregate proportions by 'CLM Classification' and the first column
aggregated_df = merged_df.groupby([first_column_name, 'CLM Classification'])['Proportion'].sum().reset_index()

# Pivot the aggregated data back to wide format
final_df = aggregated_df.pivot(index=first_column_name, columns='CLM Classification', values='Proportion').reset_index()

# Save the final aggregated data to a CSV file
final_df.to_csv(output_file, index=False)

print(f"Aggregated data saved to: {output_file}")


filtered proportions
finished merging
Aggregated data saved to: C:\Users\jayja\Documents\ArcGIS\Projects\CLM_Testing\CLM\MSE calculation\aggregated_output.csv


Add MSE Calculations

In [ ]:
# File paths
aggregated_file = r".\CLM\MSE calculation\aggregated_output.csv"
final_output_file = r".\CLM\MSE calculation\final_merged_output.csv"
mse_output_file = r".\CLM\MSE calculation\MSE_Output.csv"

# Load the CSV files
aggregated_df = pd.read_csv(aggregated_file)
final_output_df = pd.read_csv(final_output_file)

# Step 1: Find common columns (excluding 'pointid')
common_columns = list(set(aggregated_df.columns).intersection(set(final_output_df.columns)))
if 'pointid' in common_columns:
    common_columns.remove('pointid')

# Step 2: Subset DataFrames to include only common columns
aggregated_common = aggregated_df[common_columns]
final_output_common = final_output_df[common_columns]

# Step 3: Apply the MSE formula
# Scale final_merged_output values by dividing by 100
final_output_scaled = final_output_common / 100
mse_per_cell = (aggregated_common - final_output_scaled) ** 2

# Step 4: Calculate row-wise MSE
mse_per_row = mse_per_cell.sum(axis=1) / 100
aggregated_df['MSE'] = mse_per_row

# Step 5: Save the results to MSE_Output.csv
aggregated_df.to_csv(mse_output_file, index=False)

print(f"MSE results saved to: {mse_output_file}")


MSE results saved to: C:\Users\jayja\Documents\ArcGIS\Projects\CLM_Testing\CLM\MSE calculation\MSE_Output.csv


In [ ]:
# File paths
mse_output_file = r".\CLM\MSE calculation\MSE_Output.csv"
point_mse_file = r".\CLM\MSE calculation\point_MSE_V3.csv"

# Load the MSE output file
mse_df = pd.read_csv(mse_output_file)

# Extract the first and last columns
point_mse_df = mse_df.iloc[:, [0, -1]]

# Save the new DataFrame to point_MSE.csv
point_mse_df.to_csv(point_mse_file, index=False)

print(f"point_MSE.csv created with the first and last column: {point_mse_file}")

point_MSE.csv created with the first and last column: C:\Users\jayja\Documents\ArcGIS\Projects\CLM_Testing\CLM\MSE calculation\point_MSE_V3.csv
